# The minimizers of Motzkin polynomial

In [1]:
using DynamicPolynomials, MomentTools, Plots;
using JuMP, MosekTools; set_optimizer(Mosek.Optimizer); #optimizer_with_attributes(Mosek.Optimizer, "QUIET" => true);

We use Mosek solver for the convex optimization on SDP matrices.

In [2]:
X  = @polyvar x y
motz = x^4*y^2 + x^2*y^4 - 3x^2*y^2  + 1 

x⁴y² + x²y⁴ - 3x²y² + 1

This polynomial, known as Motzkin polynomial, is non-negative but not a sum of squares.

We construction a moment relaxation of order 6, imposing the moments to come from a probability measure, with total mass equal to 1:

In [3]:
v, M = minimize(motz,[],[],X,6)

Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 91              
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimizat

(nothing, A JuMP Model
Minimization problem with:
Variables: 91
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 1 constraint
`Vector{AffExpr}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: basis, degree, dual, index, moments, monomials, nu, type, variables, y)

In [4]:
v

The objective value is a lower bound of the actual mimimal value $0$ of Motzkin polynomial.

We extract a measure from the sequence of moments, using the function `MultivariateSeries.decompose`:

In [5]:
w, Xi = get_measure(M)

(Float64[], Matrix{Float64}(undef, 2, 0))

 `w` is the vector of weights and `Xi` is the matrix of points, that is support of the measure $\mu=\sum_i \omega_i \delta_{\Xi_i}$. `w[i]` is the weight of the Dirac measure corresponding to the point `Xi[:,i]` in this decomposition.

Here no point is found to approximate the optimal moment sequence.

Now, to find the minimizers, we add as constraints, that the gradient of the polynomial should vanish:

In [6]:
J = differentiate(motz,X)
v, M = minimize(motz, J,[], X, 6)

Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 91              
  Cones                  : 0               
  Scalar variables       : 73              
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective se

(3.508005206489722e-8, A JuMP Model
Minimization problem with:
Variables: 91
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 73 constraints
`Vector{AffExpr}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: basis, degree, dual, index, moments, monomials, nu, type, variables, y)

In [7]:
w, Xi = get_measure(M)

(ComplexF64[0.24999899665843472 + 2.53746840362181e-24im, 0.24999931434234496 - 2.9369695048587213e-24im, 0.24999931434234557 + 8.62076045504157e-24im, 0.24999899665843472 - 1.3649013443718234e-24im], ComplexF64[1.0000332751243977 - 1.2274562148132675e-22im -0.9999986547780946 + 3.3197704318268417e-22im 0.999998654778095 - 3.2087889721712766e-22im -1.00003327512438 + 1.1990749465519917e-22im; 0.9999937527927507 + 2.5251814192169112e-23im 1.0000004371411473 - 6.829589930462932e-23im -1.0000004371411473 + 6.601273586202847e-23im -0.9999937527927547 - 2.4667941211472628e-23im])

In [ ]:
p0 = plot([-1,-1,1,1], [-1,1,-1,1], seriestype = :scatter, color=:blue, fmt=:png)
p1 = plot!(p0,real(Xi[1,:]), real(Xi[2,:]), seriestype = :scatter, zcolor = abs.(w), m = (:heat, 0.8, Plots.stroke(1, :black)), fmt=:png)

We find approximately the $4$ minimizers.


Finally, to find the minimizers, we try another strategy and add the constraint that the polynomial should vanish, since we know that its minimal value (at the minimizers) is $0$:

In [ ]:
v, M = minimize(motz, [motz], [], X, 6)

In [ ]:
w, Xi = get_measure(M)

In [ ]:
p0 = plot([-1,-1,1,1], [-1,1,-1,1], seriestype = :scatter, color=:blue, fmt=:png)
p1 = plot!(p0,real(Xi[1,:]), real(Xi[2,:]), seriestype = :scatter, zcolor = abs.(w), m = (:heat, 0.8, Plots.stroke(1, :black)), fmt=:png)

The minimizers (in red) are close to that actual minimizers (in blue).